### Summary of findings

- Gradient Boosting doesn't seem to do better than Random Forest.
    - I might need to do more tuning of the algorithm though.
- Both can take a long time to fit with lots of data + estimators.
    - We should go over the algorithms to try to understand the $\mathcal{O}$ of them.
- 3, 8, and 9 are the most common correct labels to get mis-predicted. 9 seems like it's hard in general.

### Code

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.stats import ttest_ind
#from sklearn.cross_validation import train_test_split

In [2]:
with open('datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

If we want to save memory, we can import the values as unsigned 8-bit integers, which range from 0 to 255 - exactly the range we want. This only uses about 1/8 of the memory of the default 64-bit float, right off the bat.

At least on my computer, this helps a lot. I can't really manage to import the whole thing, and do training on it without this.

If we wanted to save more memory, we could convert to a "black and white" version, and replace the integers with 1-bit booleans.

Also, for testing, we can just import some of the rows. This will be especially useful for finding the ideal parameters for our model.

In [14]:
train=pd.read_csv(datapath+'/train.csv', nrows=5000, dtype=np.uint8)

In [15]:
train.shape

(5000, 785)

In [16]:
predictors=train.columns.drop('label')

Set up our grid search:

In [17]:
rfc=RandomForestClassifier()

In [18]:
%%time
rfc.fit(train[predictors],train['label'])

CPU times: user 259 ms, sys: 4 ms, total: 263 ms
Wall time: 263 ms


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
parameters={}

In [20]:
parameters['n_estimators']=np.linspace(5,30,num=4,dtype=np.int)
parameters['max_features']=np.linspace(0.1,1.0,num=4)
parameters['min_samples_split']=np.linspace(2,6,num=3,dtype=np.int)
parameters['min_weight_fraction_leaf']=np.linspace(0.0,0.4,num=3)

In [21]:
parameters

{'max_features': array([ 0.1,  0.4,  0.7,  1. ]),
 'min_samples_split': array([2, 4, 6]),
 'min_weight_fraction_leaf': array([ 0. ,  0.2,  0.4]),
 'n_estimators': array([ 5, 13, 21, 30])}

In [22]:
modelrfc=GridSearchCV(rfc,parameters)

The `%%time` "cell magic" command will output the amount of time it takes to run a cell.

This only times it once, so it's not necessarily representative of what it would take in the future. If you have some code you're testing that you'll loop over repeatedly, you should use the `%%timeit` (*time it*eratively) command, which will run it a bunch of times and give you statistics.

In [23]:
%%time
modelrfc.fit(train[predictors],train['label'])

CPU times: user 9min 47s, sys: 44.2 ms, total: 9min 48s
Wall time: 9min 48s


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 5, 13, 21, 30]), 'max_features': array([ 0.1,  0.4,  0.7,  1. ]), 'min_samples_split': array([2, 4, 6]), 'min_weight_fraction_leaf': array([ 0. ,  0.2,  0.4])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [25]:
modelrfc.best_params_

{'max_features': 0.10000000000000001,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 30}

We have hit our highest value for the number of estimators. This gets at a problem we may have: more estimators may be better(generally the only risk in increasing it is overfitting), but may take more time to train. Let's try a second round of searching, looking only at the number of estimators.

In [26]:
def param_dict(best):
    #outputs a parameter dictionary that can be taken in by GridSearchCV
    iterator=zip(best.keys(),best.values())
    return {i[0]:[i[1]] for i in iterator}


In [33]:
newparams=param_dict(modelrfc.best_params_)
newparams['n_estimators']=np.linspace(30, 200, num=4, dtype=int)

In [34]:
modelrfc=GridSearchCV(rfc,newparams)

In [35]:
%%time
modelrfc.fit(train[predictors],train['label'])

CPU times: user 1min 47s, sys: 32 ms, total: 1min 47s
Wall time: 1min 47s


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [0.10000000000000001], 'min_samples_split': [6], 'min_weight_fraction_leaf': [0.0], 'n_estimators': array([200, 300, 400])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [36]:
modelrfc.best_params_

{'max_features': 0.10000000000000001,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200}

In [59]:
newparams=param_dict(modelrfc.best_params_)
newparams['n_estimators']=np.linspace(200, 400, num=3, dtype=int)

In [60]:
newparams

{'max_features': [0.10000000000000001],
 'min_samples_split': [6],
 'min_weight_fraction_leaf': [0.0],
 'n_estimators': array([200, 300, 400])}

In [37]:
#%%time
#rfcRMSE=cross_val_score(modelrfc.best_estimator_, train[predictors],train['label'],cv=10)

In [39]:
#rfcRMSE.mean()

0.87463743810952921

#### Gradient Boosting

In [67]:
gbc=GradientBoostingClassifier()

In [39]:
parameters['n_estimators']=[5,10]
parameters['warm_start']=[True]

In [40]:
modelgbc=GridSearchCV(gbc,parameters)

In [41]:
%%time
modelgbc.fit(train[predictors],train['label'])

CPU times: user 29min 42s, sys: 100 ms, total: 29min 42s
Wall time: 29min 43s


GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10], 'max_features': array([ 0.1,  0.4,  0.7,  1. ]), 'min_samples_split': array([2, 4, 6]), 'min_weight_fraction_leaf': array([ 0. ,  0.2,  0.4])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [42]:
modelgbc.best_params_

{'max_features': 0.10000000000000001,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10}

In [83]:
newparamsgbc=param_dict(modelgbc.best_params_)
newparamsgbc['n_estimators']=np.linspace(500,900,num=3,dtype=int)
#newparamsgbc['max_features']=[0.05]

In [84]:
modelgbc=GridSearchCV(gbc,newparamsgbc)

In [85]:
%%time
modelgbc.fit(train[predictors],train['label'])

KeyboardInterrupt: 

In [46]:
modelgbc.best_params_

{'max_features': 0.10000000000000001,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100}

In [93]:
#newgbc=modelgbc.best_estimator_
newgbc.set_params(n_estimators=200)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=0.10000000000000001,
              max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=6,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [81]:
new_score=cross_val_score(newgbc, train[predictors],train['label'])

In [82]:
new_score.mean()-modelgbc.best_score_

-0.00040042873759205122

In [69]:
GradientBoostingClassifier?

In [73]:
newparamsgbc['n_estimators']=np.linspace(150,300,num=3,dtype=int)
newparamsgbc

{'max_features': [0.10000000000000001],
 'min_samples_split': [6],
 'min_weight_fraction_leaf': [0.0],
 'n_estimators': array([150, 225, 300])}

In [75]:
modelgbc=GridSearchCV(gbc, newparamsgbc)
modelgbc.fit(train[predictors],train['label'])
modelgbc.best_score_-new_score.mean()

0.0039958771768254797

In [76]:
modelgbc.best_params_

{'max_features': 0.10000000000000001,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300}

In [54]:
#%%time
#gbcRMSE=cross_val_score(modelgbc.best_estimator_, train[predictors],train['label'],cv=10)

CPU times: user 2min 30s, sys: 20 ms, total: 2min 30s
Wall time: 2min 30s


### Using all the data

In [88]:
train=pd.read_csv(datapath+'/train.csv',dtype=np.uint8)

In [94]:
fullgbc=newgbc
fullgbc.set_params(n_estimators=100)
fullrfc=modelrfc.best_estimator_
fullrfc.set_params(n_estimators=100)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.10000000000000001,
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
%%time
gbc_error=cross_val_score(newgbc, train[predictors],train['label'],cv=5)
rfc_error=cross_val_score(modelrfc.best_estimator_, train[predictors],train['label'],cv=5)

CPU times: user 1h 49min 55s, sys: 4min 42s, total: 1h 54min 37s
Wall time: 1h 54min 39s


In [101]:
print('Gradient booster success rate is %1.3f' % gbc_error.mean() )
print('Random forest success rate is %1.3f' % rfc_error.mean())
print('Difference = %1.3f' % (gbc_error.mean()-rfc_error.mean()))
print('p-value = %1.3E' % ttest_ind(gbc_error,rfc_error).pvalue)

Gradient booster success rate is 0.942
Random forest success rate is 0.963
Difference = -0.021
p-value = 9.938E-08


Random forest seems to do better. Considering that both are only failing to classify about 6% of the data, the 2% difference is really big. It's very much statistically significant too.

I think that we need to learn a bit more about these algorithms. I'd like to understand how computation time scales with the number of data points and the number of estimators. For the number of estimators $n$, I think it at least has to be $\mathcal{O}(n)$ for both, as part of the algorithm is just fitting the $n$ estimators independently.

In [149]:
pd.Series(rfc_error).to_csv(datapath+'/simple_random_forest_cv_error.csv')

### Failure?

I'm curious which numbers we are failing on. Let's see.

In [102]:
train['prediction']=modelrfc.best_estimator_.predict(train[predictors])

In [129]:
failures=train[(train['prediction']!=train['label'])][['label','prediction','pixel0']]

In [138]:
failed_labels=failures.groupby(['label']).count()['prediction']
failed_labels

label
0     81
1     94
2    242
3    404
4    225
5    234
6    151
7    292
8    322
9    320
Name: prediction, dtype: int64

In [140]:
failed_labels.std()

104.02163236558057

3, 8, and 9 are the labels that are most commonly mis-predicted

In [139]:
failed_predictions=failures.groupby(['prediction']).count()['label'].sort_values()
failed_predictions

prediction
7    154
1    175
6    180
0    192
4    209
3    218
5    265
8    286
2    307
9    379
Name: label, dtype: int64

In [135]:
failures.groupby(['prediction']).count()['label'].std()

71.023079191047316

The distribution of incorrect predictions is much more evenly distributed.

In [141]:
failure_counts_label=failures.groupby(['label','prediction']).count()

In [142]:
failure_counts.loc[3]

,pixel0
prediction,
0,16
1,17
2,106
4,8
5,89
6,17
7,40
8,73
9,38


3 is most commonly mis-predicted as 2 and 5.

In [143]:
failure_counts.loc[8]

,pixel0
prediction,
0,7
1,61
2,44
3,54
4,20
5,43
6,25
7,2
9,66


8 is most commonly mis-predicted as 9 or 1, but the distribution is fairly even, except it is almost never mis-predicted as 0 or 7

In [144]:
failure_counts.loc[9]

,pixel0
prediction,
0,29
1,14
2,18
3,52
4,83
5,29
6,2
7,51
8,42


9 is mis-predicted as 4 fairly often.

In [145]:
failure_counts.loc[4]

,pixel0
prediction,
0,14
1,13
2,6
3,3
5,5
6,30
7,5
8,25
9,124


Meanwhile, when 4 is mis-labeled, it is almost always mis-labeled as 9.

In [121]:
for i in xrange(10):
    print('\n correct:%i' % i)
    print(failure_counts_label.loc[i])


 correct:0
            pixel0
prediction        
2                3
3                3
4                4
5               14
6               23
7                1
8               32
9                1

 correct:1
            pixel0
prediction        
2               30
3                8
4                6
5               20
6                6
7               10
8                8
9                6

 correct:2
            pixel0
prediction        
0               29
1               10
3               38
4               34
5                9
6               38
7               40
8               31
9               13

 correct:3
            pixel0
prediction        
0               16
1               17
2              106
4                8
5               89
6               17
7               40
8               73
9               38

 correct:4
            pixel0
prediction        
0               14
1               13
2                6
3                3
5                5
6        

In [146]:
failure_counts_prediction=failures.groupby(['prediction','label']).count()

In [147]:
for i in xrange(10):
    print('\n wrong:%i' % i)
    print(failure_counts_prediction.loc[i])


 wrong:0
       pixel0
label        
2          29
3          16
4          14
5          43
6          40
7          14
8           7
9          29

 wrong:1
       pixel0
label        
2          10
3          17
4          13
5          12
6          11
7          37
8          61
9          14

 wrong:2
       pixel0
label        
0           3
1          30
3         106
4           6
5           6
6          10
7          84
8          44
9          18

 wrong:3
       pixel0
label        
0           3
1           8
2          38
4           3
5          58
7           2
8          54
9          52

 wrong:4
       pixel0
label        
0           4
1           6
2          34
3           8
5          11
6          13
7          30
8          20
9          83

 wrong:5
       pixel0
label        
0          14
1          20
2           9
3          89
4           5
6          56
8          43
9          29

 wrong:6
       pixel0
label        
0          23
1           6
2     